# Creating Prompts
With Marvin, you prompt with code to define dynamic prompts using code, eliminating the need for cumbersome template management. With this approach, you can easily create reusable and modular prompts, streamlining the development process.

In [20]:
from marvin.prompts.library import System, User, ChainOfThought
from typing import Optional


class ExpertSystem(System):
    content: str = (
        "You are a world-class expert on {{topic}}. "
        "When asked questions about {{topic}}, you answer correctly."
    )
    topic: Optional[str]


prompt = (
    ExpertSystem(topic="python")
    | User("I need to know how to write a function to find the nth Fibonacci number.")
    | ChainOfThought()  # Tell the LLM to think step by step
)

prompt.dict()

[{'role': 'system',
  'content': 'You are a world-class expert on python. When asked questions about python, you answer correctly.'},
 {'role': 'user',
  'content': 'I need to know how to write a function to find the nth Fibonacci number.'},
 {'role': 'assistant', 'content': "Let's think step by step."}]

## Templating Prompts

In many applications, templating is unavoidable. In these cases, Marvin's optional templating engine simplifies the process of sharing context across prompts to an unprecedented level. By passing native Python types or Pydantic objects into the rendering engine, you can seamlessly establish context for entire conversations. This feature enables effortless information flow and context continuity throughout the prompt interactions.

In [21]:
from marvin.prompts.library import System, User, ChainOfThought
from typing import Optional


class ExpertSystem(System):
    content: str = (
        "You are a world-class expert on {{topic}}. "
        "When asked questions about {{topic}}, you answer correctly."
    )
    topic: Optional[str]


prompt = (
    ExpertSystem()
    | User(
        "I need to know how to write a function in {{topic}} to find the nth Fibonacci"
        " number."
    )
    | ChainOfThought()  # Tell the LLM to think step by step
)

prompt.dict(topic="rust")

[{'role': 'system',
  'content': 'You are a world-class expert on rust. When asked questions about rust, you answer correctly.'},
 {'role': 'user',
  'content': 'I need to know how to write a function in rust to find the nth Fibonacci number.'},
 {'role': 'assistant', 'content': "Let's think step by step."}]

In this scenario, the prompt sets up a simulated conversation where the system establishes the user's expertise level in Python. The user then expresses their need to understand how to write a function in Python. To facilitate a step-by-step thought process, the ChainOfThought() function is used.

In [ ]:
from marvin.prompts.library import System


class ReActPattern(System):
    content = """
    You run in a loop of Thought, Action, PAUSE, Observation.
    At the end of the loop you output an Answer
    Use Thought to describe your thoughts about the question you have been asked.
    Use Action to run one of the actions available to you - then return PAUSE.
    Observation will be the result of running those actions.
  """

Marvin provides simple, opinionated components so that you can subclass and customize
prompts the same way you would for code. 

In [45]:
from marvin.prompts.library import System
import pydantic


class ColumnInfo(pydantic.BaseModel):
    name: str
    description: str


class SQLTableDescription(System):
    content = """
    If you chose to, you may query a table whose schema is defined below:
    
    {% for column in columns %}
    - {{ column.name }}: {{ column.description }}
    {% endfor %}
    """

    columns: list[ColumnInfo] = pydantic.Field(
        ..., description="name, description pairs of SQL Schema"
    )


UserQueryPrompt = SQLTableDescription(
    columns=[
        ColumnInfo(name="last_login", description="Date and time of user's last login"),
        ColumnInfo(
            name="date_created",
            description="Date and time when the user record was created",
        ),
        ColumnInfo(
            name="date_last_purchase",
            description="Date and time of user's last purchase",
        ),
    ]
)

print(UserQueryPrompt.read())

If you chose to, you may query a table whose schema is defined below:

- last_login: Date and time of user's last login
- date_created: Date and time when the user record was created
- date_last_purchase: Date and time of user's last purchase



## Rendering Prompts

A chain of prompts is turned into messages with the `render_prompts` function. This funciton has a few responsibilities in addition to generating messages: it renders templates using runtime variables, sorts messages, and trims prompts to fit into a model's context window. The last two actions depend on the optional `position` and `priority` attributes of each prompt. 

For example, the `System` prompt defines `position=0` and `priority=1` to indicate that system prompts should be rendered first and given high priority when trimming the context. (As an optimization, `render_prompt` automatically combines multiple system messages into a single message.) `ChainOfThought()` has a `position=-1` to indicate it should be the last message. If position is not set explicitly, then prompts will take the order they are added to the chain. 